In [1]:
import socket
from urllib.parse import urlparse

In [2]:
def fetch(url):
    sock = socket.socket()
    url = urlparse(url)
    host = url.netloc
    port = url.port
    if not port:
        port = 80
    sock.connect((host, port))
    request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(url.geturl(), host)
    sock.send(request.encode('ascii'))
    
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        chunk = sock.recv(4096)
    
    sock.close()
    return response

In [22]:
import sqlite3
import json
from datetime import datetime, timedelta


def datetime_gen_fn(start_date, end_date, delta=timedelta(days=1)):
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += delta


conn = sqlite3.connect('tw-axxxCulture-market.sqlite')
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS AquaticTrans;
CREATE TABLE AquaticTrans (
    id           INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    type_name    TEXT NOT NULL,
    type_code    INTEGER NOT NULL,
    market_name  TEXT NOT NULL,
    high_price   REAL NOT NULL,
    low_price    REAL NOT NULL,
    mid_price    REAL NOT NULL,
    avg_price    REAL NOT NULL,
    date         TEXT NOT NULL,
    trans_amount REAL NOT NULL
)
''')

start_date = '2016-10-04'
end_date = '2017-10-04'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')
datetime_gen = datetime_gen_fn(start_date, end_date)

for date in datetime_gen:
    roc_year = int(date.strftime('%Y')) - 1911
    query_date = '{}{}'.format(roc_year, date.strftime('%m%d'))
    url = 'http://m.coa.gov.tw/OpenData/AquaticTransData.aspx?StartDate={}&EndDate={}'.format(query_date, query_date)
    response = fetch(url)
    json_str = response.decode('utf-8').split('\r\n')[-1]
    json_parsed = json.loads(json_str)

    for item in json_parsed:
        type_name = item['魚貨名稱']
        type_code = item['品種代碼']
        market_name = item['市場名稱']
        high_price = item['上價']
        low_price = item['下價']
        mid_price = item['中價']
        avg_price = item['平均價']
        date = item['交易日期']
        trans_amount = item['交易量']

        cur.execute('''
        INSERT INTO AquaticTrans 
        (type_name, type_code, market_name, high_price, low_price, mid_price, avg_price, date, trans_amount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''', (
            type_name, type_code, market_name, high_price, low_price, mid_price, avg_price, date,
            trans_amount))

        conn.commit()

    print('{} done'.format(query_date))

conn.close()
